In [2]:
!pip install torch==2.2
!pip install torchvision==0.17
!pip install matplotlib==3.5.2
!pip install scikit-image==0.19.3
!pip install pytorch-lightning
# !pip install torchmetrics==0.11.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 918.0 kB/s eta 0:00:00 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 1.6 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.2 MB/s eta 0:00:00 MB/s eta 0:00:01


In [3]:
import os

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl

In [6]:
class ConvNet(pl.LightningModule):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.cn1 = nn.Conv2d(1, 16, 3, 1)
        self.cn2 = nn.Conv2d(16, 32, 3, 1)
        self.dp1 = nn.Dropout2d(0.10)
        self.dp2 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(4608, 64) # 4608 is basically 12 X 12 X 32
        self.fc2 = nn.Linear(64, 10)
 
    def forward(self, x):
        x = self.cn1(x)
        x = F.relu(x)
        x = self.cn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dp1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)
        x = self.fc2(x)
        op = F.log_softmax(x, dim=1)
        return op

    def training_step(self, batch, batch_num):
        train_x, train_y = batch
        y_pred = self(train_x)
        training_loss = F.cross_entropy(y_pred, train_y)
        # optional
        self.log('train_loss', training_loss, on_epoch=True, prog_bar=True)
        return training_loss

    def on_validation_epoch_end(self, outputs):
        # optional
        avg_loss = torch.stack(outputs).mean()
        return avg_loss

    def test_step(self, batch, batch_num):
        # optional
        test_x, test_y = batch
        y_pred = self(test_x)
        test_loss = F.cross_entropy(y_pred, test_y)
        # optional
        self.log('test_loss', test_loss, on_step=True, on_epoch=True, prog_bar=True)
        return test_loss

    def on_test_epoch_end(self, outputs):
        # optional
        avg_loss = torch.stack(outputs).mean()
        return avg_loss

    def configure_optimizers(self):
        return torch.optim.Adadelta(self.parameters(), lr=0.5)

    def train_dataloader(self):
        return DataLoader(MNIST(os.getcwd(), train=True, download=True, 
                                transform=transforms.Compose([transforms.ToTensor(),
                                                              transforms.Normalize((0.1302,), (0.3069,))])), 
                                batch_size=32, num_workers=4)

    def val_dataloader(self):
        # optional
        return DataLoader(MNIST(os.getcwd(), train=True, download=True, 
                                transform=transforms.Compose([transforms.ToTensor(),
                                                              transforms.Normalize((0.1302,), (0.3069,))])), 
                                batch_size=32, num_workers=4)

    def test_dataloader(self):
        # optional
        return DataLoader(MNIST(os.getcwd(), train=False, download=True, 
                                transform=transforms.Compose([transforms.ToTensor(),
                                                              transforms.Normalize((0.1302,), (0.3069,))])), 
                                batch_size=32, num_workers=4)

In [7]:
model = ConvNet()

trainer = pl.Trainer(max_epochs=10)    
trainer.fit(model)   

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name | Type      | Params | Mode 
-------------------------------------------
0 | cn1  | Conv2d    | 160    | train
1 | cn2  | Conv2d    | 4.6 K  | train
2 | dp1  | Dropout2d | 0      | train
3 | dp2  | Dropout2d | 0      | train
4 | fc1  | Linear    | 294 K  | train
5 | fc2  | Linear    | 650    | train
-------------------------------------------
300 K     Trainable params
0         Non-trainable params
300 K     Total params
1.202     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:06<00:00, 1521761.45it/s]


Extracting /home/edvard/Workspace1/155/Mastering_PyTorch/MNIST/raw/train-images-idx3-ubyte.gz to /home/edvard/Workspace1/155/Mastering_PyTorch/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 198548.28it/s]


...

Epoch 0:   0%|                                                                                                                                                                     | 0/1875 [00:00<?, ?it/s]

/home/edvard/.local/share/virtualenvs/Mastering_PyTorch-ib8cv033/lib/python3.10/site-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch 9: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:27<00:00, 67.47it/s, v_num=0, train_loss_step=0.0333, train_loss_epoch=0.0336]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 1875/1875 [00:27<00:00, 67.44it/s, v_num=0, train_loss_step=0.0333, train_loss_epoch=0.0336]


In [10]:
!pip install tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 6.5 MB/s eta 0:00:006.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 9.1 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 9.0 MB/s eta 0:00:00 MB/s eta 0:00:010:01


In [16]:
# Start tensorboard.
import os
cwd = os.getcwd()
print(cwd)
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/

/home/edvard/Workspace1/155/Mastering_PyTorch


Reusing TensorBoard on port 6006 (pid 1594469), started 0:04:58 ago. (Use '!kill 1594469' to kill it.)